# 1. 클로저 표현식

- 클로저 표현식은  고유한 익명 타입의 클로저 값을 생성합니다. 
- 클로저 유형은 캡처된 변수를 포함하는 구조체와 거의 동일합니다.

## Rust 클로저 특성
Rust 프로그래밍 언어에서 Fn, FnMut, FnOnce는 클로저와 다른 유형의 함수 객체에 대한 다양한 동작을 정의하는 특성입니다.

- Fn 특성은 여러 번 호출할 수 있고 불변으로 차용할 수 있는 클로저를 나타냅니다. 이 특성에는 self에 대한 참조를 차용하는 하나의 연관된 메서드 호출(&self, args)이 있으며, 이러한 요구 사항을 충족하는 모든 클로저에서 구현할 수 있습니다.

- FnMut 형질은 여러 번 호출할 수 있고 가변적으로 차용할 수 있는 클로저를 나타냅니다. 이 형질에는 self에 대한 가변 참조를 취하는 call_mut(&mut self, args) 메서드가 하나 있으며, 이러한 요구 사항을 충족하는 모든 클로저에 의해 구현될 수 있습니다.

- FnOnce 형질은 한 번만 호출할 수 있는 클로저를 나타냅니다. 이 특성에는 self의 소유권을 취하는 call_once(self, args) 메서드가 하나 있으며, 이러한 요구 사항을 충족하는 모든 클로저에서 구현할 수 있습니다.


## 1-1 클로저 정의 및 호출하기 

- 주의할 점은 구조체 등은 소유권이 이동하므로 클로저 내부 사용하면 예외 발생 

### 클로저 정의 

-  매개변수는 |매개변수 | 정의하고 
-  반환자료형은  -> 다음에 정의
-  본문은 { } 사이에 정의한다. 

In [9]:
fn main() {
    let m = 1.0;
    let c = 2.0;
    let _z = String::from("가가가"); 

    let line = |x:f64|->f64 {      // 변수에 클로저 할당할 때는 main 함수 내부에 정의 할 것
        let s = m*x + c;           // 함수의 지역변수를 클로저에서 사용이 가능 : 정수는 이동되지 않아서 사용가능 
        //let _zz = _z;             // _z는 한번 소유권 이동이 되어서 계속 호출시 문제됨 ...
        s                          // 내부에서 소유권 변동을 주의해야 함  
    };
    println!("{}", line(0.0));
    println!("m {:.2} ", m);
    //println!(" z { } ", _z);
}

main();

2
m 1.00 


## 1-2 클로저 타입 알아보기

- 클로저는 변수, 매개변수, 함수 반환으로 사용할 수 있다.


- https://doc.rust-lang.org/reference/types/closure.html



## 클로저가 환경에서 값을 캡처하는 세 가지 방식Permalink
소유권 받기, 불변으로 빌려오기, 가변으로 빌려오는 방식이다.

FnOnce는 캡처한 변수의 소유권을 클로저 안으로 옮긴다. 클로저가 동일한 변수에 대해 한 번만 소유권을 얻을 수 있어서 Once라는 이름이 들어갔다.

Fn은 캡처한 값을 불변으로 빌려온다.

FnMut은 값을 가변으로 빌려와서 환경을 변경할 수 있다.

## 일반 클로저를 변수에 할당

- 함수포인터로 지정해서 변수에 할당한다.


In [11]:
fn main() {
    let add_one_v2: fn(u32)->u32 = |x: u32| -> u32 { x + 1 };
    
    println!(" {} ", add_one_v2(100));
    
}

main();

 101 


### 캡처된 자유변수가 있을 경우

- 일반적인 익명함수가 아니라서 함수포인터로 지정할 수 없다 

In [12]:
fn main() {
    let cap_var = 300; 
    let add_one_v2: fn(u32)->u32 = |x: u32| -> u32 { x + cap_var};
    
    println!(" {} ", add_one_v2(100));
    
}

main();

Error: mismatched types

### 함수 트레이트로 정의

- 트레이트 객체로 처리하라고 에러 표시 

In [14]:
fn main() {
    let cap_var = 300; 
    let add_one_v2: dyn FnOnce(u32)->u32 = |x: u32| -> u32 { x + cap_var};
    
    println!(" {} ", add_one_v2(100));
    
}

main();

Error: mismatched types

Error: the size for values of type `dyn FnOnce(u32) -> u32` cannot be known at compilation time

## 1-3 위의 문제를 해결하기 하려면 

- 트레이트 객체를 박스로 지정해서 처리 

### 트레이트 객체 (Trait Object):

- 트레이트 객체는 구체적인 타입이 아닌 트레이트 자체를 나타내는 타입입니다.
- 트레이트 객체는 dyn 키워드와 함께 사용되며, 동적 디스패치를 통해 다양한 타입을 처리할 수 있습니다.
- 트레이트 객체는 런타임에 해당하는 타입을 알아내기 위해 가상 함수 테이블(virtual function table)을 사용합니다.

In [5]:
fn main() {
    let cap_var = 300; 
    let add_one_v2: Box<dyn Fn(u32)->u32> = Box::new(|x: u32| -> u32 { x });
    
    println!(" {} ", add_one_v2(100));
    
}

main();

 100 


### 타입을 처리하는 함수 정의 

In [7]:
use std::any::type_name;

fn type_of<T>(_: T) -> &'static str {
    type_name::<T>()
}

### 캡처하는 자유변수에 대한 소유권 이동 

- move 키워드로 캡처값의 소유권을 강제로 갖기

In [8]:
fn main() {
    let x = vec![1, 2, 3];

    let equal_to_x = move |z| z == x;

    println!("{:?}", type_of(&equal_to_x));

    let y = vec![1, 2, 3];

    assert!(equal_to_x(y));
}

main();

"&ctx::main::{{closure}}"


## 1-4 클로저 축약형으로 작성하기 

- 클로저를 정의할 때 매개변수의 자료형이나 반환값 등을 생략할 수 있다.
- 또한 한줄만 본문이 작성할 경우는 중괄호도 생략이 가능하다

### 함수와 클러저 정의 알아보기

- 함수 정의를 클로저로 변경하기
- 클로저는 이름이 없어서 활용하려면 변수에 할당해서 사용한다. 

In [10]:
fn main() {
    fn  add_one_v1   (x: u32) -> u32 { x + 1 }   // 함수 정의 
    
    let add_one_v2 = |x: u32| -> u32 { x + 1 };  // 클로저 정의 
    let add_one_v3 = |x|             { x + 1 };  // 클로저 정의 : 타입 에노테이션 생략 
    let add_one_v4 = |x|               x + 1  ;  // 하나의 문장이라 표현식으로 지정 
    println!(" add_one_v1 {} ", add_one_v1(100));
    println!(" add_one_v2 {} ", add_one_v2(100));
    println!(" add_one_v3 {} ", add_one_v3(100));
    println!(" add_one_v4 {} ", add_one_v4(100));
}

In [11]:
main()

 add_one_v1 101 
 add_one_v2 101 
 add_one_v3 101 
 add_one_v4 101 


()

### 클로저의 매개변수에 자료형 지정하기
- 매개변수의 자료형이 불명확할 때는 반드시 지정해줘야 한다. 

In [12]:
fn main() {
    let c  = |x:u32| {x+x};

    println!(" {} ",c(10))
}

In [13]:
main()

 20 


()

## 1-5 구조체 내부에 클로저 정의하기

### 두 개의 클로저를 정의하고 하나를 다른 클로저에서 처리하기 

In [14]:
fn main() {
    let c  = |x:u32| {x+x};
    let d  = |y| c(y);

    println!(" {} ",d(10))
}

In [15]:
main();

 20 


### 클로저를 반환할 때는 캡처된 변수를 이동시키기

In [16]:
fn main() {
    
    let d  = |y| {move |x| x+y};

    println!(" {} ",d(10)(20))
}

In [17]:
main();

 30 


## 1-6  스마트 포인터로 클로저 반환하기 

- 트레이트 객체는 동적으로 처리하므로 스마트 포인터와 함게 사용해서 처리 

In [15]:
fn main() {
    let f: Box<_> = Box::new(|| {println!("foo")});
    f();
    let fone: Box<dyn Fn(u32)> = Box::new(|x| {println!("foo {}",x)});
    fone(100);
    let ftwo: Box<dyn Fn(u32,u32)> = Box::new(|x,y| {println!("foo {}",x+y)});
    ftwo(100,200);
}

main();

foo
foo 100
foo 300


## 1-7 다양한 타입 처리 알아보기

In [24]:
fn main() {
    let x = 100; 
    fn add() {}
    let v_char = vec!['A','c','e'];
    let v_num = vec![1,2];
    let clo :fn()->u32 = || {100 };
    let fun : fn() = add;
    let clo_mo : Box<dyn Fn() -> i32> = Box::new(move || { 100 + x});

    print_variable_type(&v_char);
    print_variable_type(&v_char[0]);
    print_variable_type(&v_num[0]);   
    print_variable_type(&clo);  
    print_variable_type(&fun); 
    print_variable_type(&clo_mo);
}

fn print_variable_type<K: ?Sized>(_: &K) {
    println!("{:?}", std::any::type_name::<K>())
}

main();

"alloc::vec::Vec<char>"
"char"
"i32"
"fn() -> u32"
"fn()"
"alloc::boxed::Box<dyn core::ops::function::Fn() -> i32>"
